### Generating Molecule Information

The notebook generates different `csv` files containing information about the hydrocarbon (pseudo-)molecules in 2 Carbon tanks connected via a Carbon nanotube. The information generated by the notebook include:

* the x, y, and z coordinates of the (pseudo-)molecules
* the bonds information of the (pseudo-)molecules
* the angles information of the (pseudo-)molecules
* the dihedrals information of the (pseudo-)molecules

As an input, the notebook accepts:

* `num_of_mol` = the total number of (pseduo-)molecules in each tank
* `num_of_comp` = the number of hydrocarbon components in the hydrocarbon mixture
* `composition` = a list containing the % composition of each of the components of the hydrocarbon mixture. The length of this list should be equal to `num_of_comp`
* `num_dec` = the number of decimal places for expressing the coordinates
* `nC1`, `nC2`, ... = the number of Carbon atoms in 1st, 2nd, ... components of the hydrocarbon mixture
* `xlo`,`xhi`,`ylo`,`yhi`,`zlo`,`zhi` = coordinates defining the LAMMPS simulation box
* `x1`,`x2`,`y1`,`y2`,`z1`,`z2` = coordinates defining the edges of the initial positions of the (pseudo-)molecules in a single tank
* `nan_pore_len` = the length (in angstoms) of the nanotube connecting the 2 tanks
* `filepath` = the directory to contain the output files
* `C_out_filename` = the directory of the output `csv` file containing the co-ordinates of the (pseudo-)molecules
* `bonds_out_filename` = the directory of the output `csv` file containing the bonds information of the (pseudo-)molecules
* `angles_out_filename` = the directory of the output `csv` file containing the angles information of the (pseudo-)molecules
* `dihedrals_out_filename` = the directory of the output `csv` file containing the dihedrals information of the (pseudo-)molecules



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys
import random
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
num_of_mol = 

num_of_comp = 

composition = []

num_dec = 

comp_norm = [round((comp/100)*num_of_mol) for comp in composition]

atom_count = []

nC1, nC2, nC3, nC4, nC5 = 

xlo, xhi = 
ylo, yhi = 
zlo, zhi =

x1, x2 =
y1, y2 =
z1, z2 =

nan_pore_len =

In [ ]:
filepath = Path(r'C:\Users\malomo.mg\Documents\Python\Files')
C_out_filename = filepath/f'mols_{num_of_mol}x2.csv'
bonds_out_filename = filepath/f'bonds_{num_of_mol}x2.csv'
angles_out_filename = filepath/f'angles_{num_of_mol}x2.csv'
dihedrals_out_filename = filepath/f'dihedrals_{num_of_mol}x2.csv'

In [ ]:
offset = y2-y1+nan_pore_len

In [ ]:
def coordinates (num, C_no, X1, X2, Y1, Y2, Z1, Z2):

  offset = 0.2
  x = np.arange(X1+abs(offset*X1), X2-abs(offset*X2), 1)
  y = np.arange(Y1+abs(offset*Y1), (((Y2-abs(offset*Y2))-(Y1+abs(offset*Y1)))//C_no)*C_no+(Y1+abs(offset*Y1)), 1)
  z = random.uniform(Z1, Z2)
  n = 0
  coord = [[], [], []]
  row = 0

  while True:
    z = z+row 
    for i in x:
      for j in y:
        coord[0].append(round(i,num_dec))
        coord[1].append(round(j,num_dec))
        coord[2].append(round(z,num_dec))
        n = n+1
        if n == num:
          break
      else:
        continue
      break

    row = row + 1

    if n == num:
      break 
    
  return coord

In [ ]:
# atom_type 2 = CH4
C1 = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

a_id = list(range(1, (comp_norm[0]*nC1)+1))

C1.atom_id = a_id
C1.molecule_id = a_id
C1 = C1.assign(atom_type = 2)
C1 = C1.assign(charge = 0)

coord_C1 = coordinates(comp_norm[0]*nC1, nC1, x1, x2, y1, y2, 0, z2/len(composition))

C1.x_pos, C1.y_pos, C1.z_pos = coord_C1[0], coord_C1[1], coord_C1[2]

atom_count.append(C1.shape[0])

C1_shape = atom_count[-1]

In [ ]:
C1_copy = C1.copy()

C1_copy.y_pos = C1_copy.y_pos - offset

C1_copy.atom_id, C1_copy.molecule_id = C1_copy.atom_id+C1_shape, C1_copy.molecule_id+C1_shape

In [ ]:
# atom_type 3 = CH3
C2 = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

a_id = list(range(1, (comp_norm[1]*nC2)+1))

C2.atom_id = a_id
C2 = C2.assign(atom_type = 3)
C2 = C2.assign(charge = 0)
C2.loc[0, 'molecule_id'] = 1

for i in range(1, (comp_norm[1]*nC2)):
  if C2.loc[i, 'atom_id']%2 == 0:
    C2.loc[i, 'molecule_id'] = C2.loc[i, 'atom_id']/2   
  else:
    C2.loc[i, 'molecule_id'] = C2.loc[i-1, 'molecule_id']+1
    
coord_C2 = coordinates(comp_norm[1]*nC2, nC2, x1, x2, y1, y2, z2/len(composition), z2*2/len(composition))

C2.x_pos, C2.y_pos, C2.z_pos = coord_C2[0], coord_C2[1], coord_C2[2]

atom_count.append(C2.shape[0])

C2_shape = atom_count[-1]

C2.atom_id = C2.atom_id + (C1_shape*2)

C2.molecule_id = C2.molecule_id + (C1_shape*2)

In [ ]:
C2_copy = C2.copy()

C2_copy.y_pos = C2_copy.y_pos - offset

C2_copy.atom_id, C2_copy.molecule_id = C2_copy.atom_id+C2_shape, C2_copy.molecule_id+(C2_shape/nC2)

In [ ]:
# atom_type 3 = CH3
# atom_type 4 = CH2
C5 = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

a_id = list(range(1, (comp_norm[2]*nC3)+1))

C5.atom_id = a_id
C5 = C5.assign(charge = 0)
C5.loc[0, 'molecule_id'] = 1
C5.loc[0, 'atom_type'] = 3

for i in range(1, comp_norm[2]*nC3):
  if ((C5.loc[i, 'atom_id']%nC3 > 1) and (C5.loc[i, 'atom_id']%nC3 <= (nC3-1))) or (C5.loc[i, 'atom_id']%nC3 == 0):
    C5.loc[i, 'molecule_id'] = C5.loc[i-1, 'molecule_id']
  else:
    C5.loc[i, 'molecule_id'] = C5.loc[i-1, 'molecule_id']+1

  if (C5.loc[i, 'atom_id']%nC3 == 1) or (C5.loc[i, 'atom_id']%nC3 == 0):
    C5.loc[i, 'atom_type'] = 3
  else:
    C5.loc[i, 'atom_type'] = 4

coord_C5 = coordinates(comp_norm[2]*nC3, nC3, x1, x2, y1, y2, z2*2/len(composition), z2*3/len(composition))

C5.x_pos, C5.y_pos, C5.z_pos = coord_C5[0], coord_C5[1], coord_C5[2]

atom_count.append(C5.shape[0])

C5_shape = atom_count[-1]

C5.atom_id = C5.atom_id + C2_copy.atom_id.max()

C5.molecule_id = C5.molecule_id + C2_copy.molecule_id.max()

In [ ]:
C5_copy = C5.copy()

C5_copy.y_pos = C5_copy.y_pos - offset

C5_copy.atom_id, C5_copy.molecule_id = C5_copy.atom_id+C5_shape, C5_copy.molecule_id+(C5_shape/nC3)

In [ ]:
# atom_type 3 = CH3
# atom_type 4 = CH2
C7 = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

a_id = list(range(1, (comp_norm[3]*nC4)+1))

C7.atom_id = a_id
C7 = C7.assign(charge = 0)
C7.loc[0, 'molecule_id'] = 1
C7.loc[0, 'atom_type'] = 3

for i in range(1, (comp_norm[3]*nC4)):
  if ((C7.loc[i, 'atom_id']%nC4 > 1) and (C7.loc[i, 'atom_id']%nC4 <= (nC4-1))) or (C7.loc[i, 'atom_id']%nC4 == 0):
    C7.loc[i, 'molecule_id'] = C7.loc[i-1, 'molecule_id']
  else:
    C7.loc[i, 'molecule_id'] = C7.loc[i-1, 'molecule_id']+1

  if (C7.loc[i, 'atom_id']%nC4 == 1) or (C7.loc[i, 'atom_id']%nC4 == 0):
    C7.loc[i, 'atom_type'] = 3
  else:
    C7.loc[i, 'atom_type'] = 4

coord_C7 = coordinates(comp_norm[3]*nC4, nC4, x1, x2, y1, y2, z2*3/len(composition), z2*4/len(composition))

C7.x_pos, C7.y_pos, C7.z_pos = coord_C7[0], coord_C7[1], coord_C7[2]

atom_count.append(C7.shape[0])

C7_shape = atom_count[-1]

C7.atom_id = C7.atom_id + C5_copy.atom_id.max()

C7.molecule_id = C7.molecule_id + C5_copy.molecule_id.max()

In [ ]:
C7_copy = C7.copy()

C7_copy.y_pos = C7_copy.y_pos - offset

C7_copy.atom_id, C7_copy.molecule_id = C7_copy.atom_id+C7_shape, C7_copy.molecule_id+(C7_shape/nC4)

In [ ]:
# atom_type 3 = CH3
# atom_type 4 = CH2
C13 = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

a_id = list(range(1, (comp_norm[4]*nC5)+1))

C13.atom_id = a_id
C13 = C13.assign(charge = 0)
C13.loc[0, 'molecule_id'] = 1

#C13.loc[0, 'x_pos'], C13.loc[0, 'y_pos'], C13.loc[0, 'z_pos'] = x, y, z
C13.loc[0, 'atom_type'] = 3

for i in range(1, comp_norm[4]*nC5):
  if ((C13.loc[i, 'atom_id']%nC5 > 1) and (C13.loc[i, 'atom_id']%nC5 <= (nC5-1))) or (C13.loc[i, 'atom_id']%nC5 == 0):
    C13.loc[i, 'molecule_id'] = C13.loc[i-1, 'molecule_id']
  else:
    C13.loc[i, 'molecule_id'] = C13.loc[i-1, 'molecule_id']+1

  if (C13.loc[i, 'atom_id']%nC5 == 1) or (C13.loc[i, 'atom_id']%nC5 == 0):
    C13.loc[i, 'atom_type'] = 3
  else:
    C13.loc[i, 'atom_type'] = 4

coord_C13 = coordinates(comp_norm[4]*nC5, nC5, x1, x2, y1, y2, z2*4/len(composition), z2*5/len(composition))

C13.x_pos, C13.y_pos, C13.z_pos = coord_C13[0], coord_C13[1], coord_C13[2]

atom_count.append(C13.shape[0])

C13_shape = atom_count[-1]

C13.atom_id = C13.atom_id + C7_copy.atom_id.max()

C13.molecule_id = C13.molecule_id + C7_copy.molecule_id.max()

In [ ]:
C13_copy = C13.copy()

C13_copy.y_pos = C13_copy.y_pos - offset

C13_copy.atom_id, C13_copy.molecule_id = C13_copy.atom_id+C13_shape, C13_copy.molecule_id+(C13_shape/nC5)

In [ ]:
num_atoms = [num*2 for num in atom_count]

num_atoms

[739512, 372776, 301130, 221312, 289328]

In [ ]:
np.cumsum(num_atoms)

array([ 739512, 1112288, 1413418, 1634730, 1924058], dtype=int32)

In [ ]:
# bond type 1 - CH3-CH3
# bond type 2 - CH3-CH2
# bond type 3 - CH2-CH2

C2_bonds = pd.DataFrame(columns = ['bond_id', 'bond_type', 'atom_id1', 'atom_id2'])

i = 0

for j in range(0, C2_shape):
  if (C2.molecule_id.iloc[j] == C2.molecule_id.iloc[j-1]) and (C2.atom_type.iloc[j] == 3 and C2.atom_type.iloc[j-1] == 3):
    C2_bonds.loc[i, 'bond_type'] = 1
    C2_bonds.loc[i, 'atom_id1'] = C2.atom_id.iloc[j-1]
    C2_bonds.loc[i, 'atom_id2'] = C2.atom_id.iloc[j]
    i = i+1
  elif (C2.molecule_id.iloc[j] == C2.molecule_id.iloc[j-1]) and ((C2.atom_type.iloc[j] == 3 and C2.atom_type.iloc[j-1] == 4) or (C2.atom_type.iloc[j] == 4 and C2.atom_type.iloc[j-1] == 3)):
    C2_bonds.loc[i, 'bond_type'] = 2
    C2_bonds.loc[i, 'atom_id1'] = C2.atom_id.iloc[j-1]
    C2_bonds.loc[i, 'atom_id2'] = C2.atom_id.iloc[j]
    i = i+1
  elif (C2.molecule_id.iloc[j] == C2.molecule_id.iloc[j-1]) and (C2.atom_type.iloc[j] == 4 and C2.atom_type.iloc[j-1] == 4):
    C2_bonds.loc[i, 'bond_type'] = 3
    C2_bonds.loc[i, 'atom_id1'] = C2.atom_id.iloc[j-1]
    C2_bonds.loc[i, 'atom_id2'] = C2.atom_id.iloc[j]
    i = i+1

b_id = list(range(1, C2_bonds.shape[0]+1))
C2_bonds.bond_id = b_id

In [ ]:
C2_bonds_copy = C2_bonds.copy()

C2_bonds_copy.bond_id = C2_bonds_copy.bond_id+C2_bonds.bond_id.max()

C2_bonds_copy.atom_id1, C2_bonds_copy.atom_id2 = C2_bonds_copy.atom_id1+C2_shape, C2_bonds_copy.atom_id2+C2_shape

In [ ]:
C2_bonds = pd.concat([C2_bonds, C2_bonds_copy]).reset_index(drop = True)

del C2_bonds_copy

In [ ]:
# bond type 1 - CH3-CH3
# bond type 2 - CH3-CH2
# bond type 3 - CH2-CH2

C5_bonds = pd.DataFrame(columns = ['bond_id', 'bond_type', 'atom_id1', 'atom_id2'])

i = 0

for j in range(0, C5_shape):
  if (C5.molecule_id.iloc[j] == C5.molecule_id.iloc[j-1]) and (C5.atom_type.iloc[j] == 3 and C5.atom_type.iloc[j-1] == 3):
    C5_bonds.loc[i, 'bond_type'] = 1
    C5_bonds.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-1]
    C5_bonds.loc[i, 'atom_id2'] = C5.atom_id.iloc[j]
    i = i+1
  elif (C5.molecule_id.iloc[j] == C5.molecule_id.iloc[j-1]) and ((C5.atom_type.iloc[j] == 3 and C5.atom_type.iloc[j-1] == 4) or (C5.atom_type.iloc[j] == 4 and C5.atom_type.iloc[j-1] == 3)):
    C5_bonds.loc[i, 'bond_type'] = 2
    C5_bonds.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-1]
    C5_bonds.loc[i, 'atom_id2'] = C5.atom_id.iloc[j]
    i = i+1
  elif (C5.molecule_id.iloc[j] == C5.molecule_id.iloc[j-1]) and (C5.atom_type.iloc[j] == 4 and C5.atom_type.iloc[j-1] == 4):
    C5_bonds.loc[i, 'bond_type'] = 3
    C5_bonds.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-1]
    C5_bonds.loc[i, 'atom_id2'] = C5.atom_id.iloc[j]
    i = i+1

b_id = list(range(1, C5_bonds.shape[0]+1))
C5_bonds.bond_id = b_id
C5_bonds.bond_id = C5_bonds.bond_id + C2_bonds.bond_id.max()

In [ ]:
C5_bonds_copy = C5_bonds.copy()

C5_bonds_copy.bond_id = C5_bonds_copy.bond_id+C5_bonds.shape[0]

C5_bonds_copy.atom_id1, C5_bonds_copy.atom_id2 = C5_bonds_copy.atom_id1+C5_shape, C5_bonds_copy.atom_id2+C5_shape

In [ ]:
C5_bonds = pd.concat([C5_bonds, C5_bonds_copy]).reset_index(drop = True)

del C5_bonds_copy

In [ ]:
# bond type 1 - CH3-CH3
# bond type 2 - CH3-CH2
# bond type 3 - CH2-CH2

C7_bonds = pd.DataFrame(columns = ['bond_id', 'bond_type', 'atom_id1', 'atom_id2'])

i = 0

for j in range(0, C7_shape):
  if (C7.molecule_id.iloc[j] == C7.molecule_id.iloc[j-1]) and (C7.atom_type.iloc[j] == 3 and C7.atom_type.iloc[j-1] == 3):
    C7_bonds.loc[i, 'bond_type'] = 1
    C7_bonds.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-1]
    C7_bonds.loc[i, 'atom_id2'] = C7.atom_id.iloc[j]
    i = i+1
  elif (C7.molecule_id.iloc[j] == C7.molecule_id.iloc[j-1]) and ((C7.atom_type.iloc[j] == 3 and C7.atom_type.iloc[j-1] == 4) or (C7.atom_type.iloc[j] == 4 and C7.atom_type.iloc[j-1] == 3)):
    C7_bonds.loc[i, 'bond_type'] = 2
    C7_bonds.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-1]
    C7_bonds.loc[i, 'atom_id2'] = C7.atom_id.iloc[j]
    i = i+1
  elif (C7.molecule_id.iloc[j] == C7.molecule_id.iloc[j-1]) and (C7.atom_type.iloc[j] == 4 and C7.atom_type.iloc[j-1] == 4):
    C7_bonds.loc[i, 'bond_type'] = 3
    C7_bonds.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-1]
    C7_bonds.loc[i, 'atom_id2'] = C7.atom_id.iloc[j]
    i = i+1

b_id = list(range(1, C7_bonds.shape[0]+1))
C7_bonds.bond_id = b_id
C7_bonds.bond_id = C7_bonds.bond_id + C5_bonds.bond_id.max()

In [ ]:
C7_bonds_copy = C7_bonds.copy()

C7_bonds_copy.bond_id = C7_bonds_copy.bond_id+C7_bonds.shape[0]

C7_bonds_copy.atom_id1, C7_bonds_copy.atom_id2 = C7_bonds_copy.atom_id1+C7_shape, C7_bonds_copy.atom_id2+C7_shape

In [ ]:
C7_bonds = pd.concat([C7_bonds, C7_bonds_copy]).reset_index(drop = True)

del C7_bonds_copy

In [ ]:
# bond type 1 - CH3-CH3
# bond type 2 - CH3-CH2
# bond type 3 - CH2-CH2

C13_bonds = pd.DataFrame(columns = ['bond_id', 'bond_type', 'atom_id1', 'atom_id2'])

i = 0

for j in range(0, C13_shape):
  if (C13.molecule_id.iloc[j] == C13.molecule_id.iloc[j-1]) and (C13.atom_type.iloc[j] == 3 and C13.atom_type.iloc[j-1] == 3):
    C13_bonds.loc[i, 'bond_type'] = 1
    C13_bonds.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-1]
    C13_bonds.loc[i, 'atom_id2'] = C13.atom_id.iloc[j]
    i = i+1
  elif (C13.molecule_id.iloc[j] == C13.molecule_id.iloc[j-1]) and ((C13.atom_type.iloc[j] == 3 and C13.atom_type.iloc[j-1] == 4) or (C13.atom_type.iloc[j] == 4 and C13.atom_type.iloc[j-1] == 3)):
    C13_bonds.loc[i, 'bond_type'] = 2
    C13_bonds.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-1]
    C13_bonds.loc[i, 'atom_id2'] = C13.atom_id.iloc[j]
    i = i+1
  elif (C13.molecule_id.iloc[j] == C13.molecule_id.iloc[j-1]) and (C13.atom_type.iloc[j] == 4 and C13.atom_type.iloc[j-1] == 4):
    C13_bonds.loc[i, 'bond_type'] = 3
    C13_bonds.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-1]
    C13_bonds.loc[i, 'atom_id2'] = C13.atom_id.iloc[j]
    i = i+1

b_id = list(range(1, C13_bonds.shape[0]+1))
C13_bonds.bond_id = b_id
C13_bonds.bond_id = C13_bonds.bond_id + C7_bonds.bond_id.max()

In [ ]:
C13_bonds_copy = C13_bonds.copy()

C13_bonds_copy.bond_id = C13_bonds_copy.bond_id+C13_bonds.shape[0]

C13_bonds_copy.atom_id1, C13_bonds_copy.atom_id2 = C13_bonds_copy.atom_id1+C13_shape, C13_bonds_copy.atom_id2+C13_shape

In [ ]:
C13_bonds = pd.concat([C13_bonds, C13_bonds_copy]).reset_index(drop = True)

del C13_bonds_copy

In [ ]:
C_bonds = pd.concat([C2_bonds, C5_bonds]).reset_index(drop = True)
C_bonds = pd.concat([C_bonds, C7_bonds]).reset_index(drop = True)
C_bonds = pd.concat([C_bonds, C13_bonds]).reset_index(drop = True)

del C2_bonds, C5_bonds, C7_bonds, C13_bonds

In [ ]:
# angle type 1 - CH3-CH2-CH3  CH3-CH2-CH2  CH2-CH2-CH2

C5_angles = pd.DataFrame(columns = ['angle_id', 'angle_type', 'atom_id1', 'atom_id2', 'atom_id3'])

i = 0

for j in range(0, C5_shape):
  if (C5.molecule_id.iloc[j] == C5.molecule_id.iloc[j-1] == C5.molecule_id.iloc[j-2]):
    if (((C5.atom_type.iloc[j] == C5.atom_type.iloc[j-2] == 3) and C5.atom_type.iloc[j-1] == 4) or ((C5.atom_type.iloc[j] == C5.atom_type.iloc[j-1] == 4) and C5.atom_type.iloc[j-2] == 3) or ((C5.atom_type.iloc[j-2] == C5.atom_type.iloc[j-1] == 4) and C5.atom_type.iloc[j] == 3) or (C5.atom_type.iloc[j] == C5.atom_type.iloc[j-1] == C5.atom_type.iloc[j-2] == 4)):
      C5_angles.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-2]
      C5_angles.loc[i, 'atom_id2'] = C5.atom_id.iloc[j-1]
      C5_angles.loc[i, 'atom_id3'] = C5.atom_id.iloc[j]
      i = i+1

ang_id = list(range(1, C5_angles.shape[0]+1))
C5_angles.angle_id = ang_id

C5_angles = C5_angles.assign(angle_type = 1)

In [ ]:
C5_angles_copy = C5_angles.copy()

C5_angles_copy.angle_id = C5_angles_copy.angle_id+C5_angles.shape[0]

C5_angles_copy.atom_id1, C5_angles_copy.atom_id2, C5_angles_copy.atom_id3 = C5_angles_copy.atom_id1+C5_shape, C5_angles_copy.atom_id2+C5_shape, C5_angles_copy.atom_id3+C5_shape

In [ ]:
C5_angles = pd.concat([C5_angles, C5_angles_copy]).reset_index(drop = True)

del C5_angles_copy

In [ ]:
# angle type 1 - CH3-CH2-CH3  CH3-CH2-CH2  CH2-CH2-CH2

C7_angles = pd.DataFrame(columns = ['angle_id', 'angle_type', 'atom_id1', 'atom_id2', 'atom_id3'])

i = 0

for j in range(0, C7_shape):
  if (C7.molecule_id.iloc[j] == C7.molecule_id.iloc[j-1] == C7.molecule_id.iloc[j-2]):
    if (((C7.atom_type.iloc[j] == C7.atom_type.iloc[j-2] == 3) and C7.atom_type.iloc[j-1] == 4) or ((C7.atom_type.iloc[j] == C7.atom_type.iloc[j-1] == 4) and C7.atom_type.iloc[j-2] == 3) or ((C7.atom_type.iloc[j-2] == C7.atom_type.iloc[j-1] == 4) and C7.atom_type.iloc[j] == 3) or (C7.atom_type.iloc[j] == C7.atom_type.iloc[j-1] == C7.atom_type.iloc[j-2] == 4)):
      C7_angles.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-2]
      C7_angles.loc[i, 'atom_id2'] = C7.atom_id.iloc[j-1]
      C7_angles.loc[i, 'atom_id3'] = C7.atom_id.iloc[j]
      i = i+1

ang_id = list(range(1, C7_angles.shape[0]+1))
C7_angles.angle_id = ang_id
C7_angles.angle_id = C7_angles.angle_id + C5_angles.angle_id.max()

C7_angles = C7_angles.assign(angle_type = 1)

In [ ]:
C7_angles_copy = C7_angles.copy()

C7_angles_copy.angle_id = C7_angles_copy.angle_id+C7_angles.shape[0]

C7_angles_copy.atom_id1, C7_angles_copy.atom_id2, C7_angles_copy.atom_id3 = C7_angles_copy.atom_id1+C7_shape, C7_angles_copy.atom_id2+C7_shape, C7_angles_copy.atom_id3+C7_shape

In [ ]:
C7_angles = pd.concat([C7_angles, C7_angles_copy]).reset_index(drop = True)

del C7_angles_copy

In [ ]:
# angle type 1 - CH3-CH2-CH3  CH3-CH2-CH2  CH2-CH2-CH2

C13_angles = pd.DataFrame(columns = ['angle_id', 'angle_type', 'atom_id1', 'atom_id2', 'atom_id3'])

i = 0

for j in range(0, C13_shape):
  if (C13.molecule_id.iloc[j] == C13.molecule_id.iloc[j-1] == C13.molecule_id.iloc[j-2]):
    if (((C13.atom_type.iloc[j] == C13.atom_type.iloc[j-2] == 3) and C13.atom_type.iloc[j-1] == 4) or ((C13.atom_type.iloc[j] == C13.atom_type.iloc[j-1] == 4) and C13.atom_type.iloc[j-2] == 3) or ((C13.atom_type.iloc[j-2] == C13.atom_type.iloc[j-1] == 4) and C13.atom_type.iloc[j] == 3) or (C13.atom_type.iloc[j] == C13.atom_type.iloc[j-1] == C13.atom_type.iloc[j-2] == 4)):
      C13_angles.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-2]
      C13_angles.loc[i, 'atom_id2'] = C13.atom_id.iloc[j-1]
      C13_angles.loc[i, 'atom_id3'] = C13.atom_id.iloc[j]
      i = i+1

ang_id = list(range(1, C13_angles.shape[0]+1))
C13_angles.angle_id = ang_id
C13_angles.angle_id = C13_angles.angle_id + C7_angles.angle_id.max()

C13_angles = C13_angles.assign(angle_type = 1)

In [ ]:
C13_angles_copy = C13_angles.copy()

C13_angles_copy.angle_id = C13_angles_copy.angle_id+C13_angles.shape[0]

C13_angles_copy.atom_id1, C13_angles_copy.atom_id2, C13_angles_copy.atom_id3 = C13_angles_copy.atom_id1+C13_shape, C13_angles_copy.atom_id2+C13_shape, C13_angles_copy.atom_id3+C13_shape

In [ ]:
C13_angles = pd.concat([C13_angles, C13_angles_copy]).reset_index(drop = True)

del C13_angles_copy

In [ ]:
C_angles = pd.concat([C5_angles, C7_angles]).reset_index(drop = True)
C_angles = pd.concat([C_angles, C13_angles]).reset_index(drop = True)

del C5_angles, C7_angles, C13_angles

In [ ]:
# dihedral type 1 - CH3-CH2-CH2-CH3  CH2-CH2-CH2-CH2  CH3-CH2-CH2-CH2

C5_dihedrals = pd.DataFrame(columns = ['dihedral_id', 'dihedral_type', 'atom_id1', 'atom_id2', 'atom_id3', 'atom_id4'])

i = 0

for j in range(0, C5_shape):
  if (C5.molecule_id.iloc[j] == C5.molecule_id.iloc[j-1] == C5.molecule_id.iloc[j-2] == C5.molecule_id.iloc[j-3]):
    if ((C5.atom_type.iloc[j] == C5.atom_type.iloc[j-3] == 3) and (C5.atom_type.iloc[j-1] == C5.atom_type.iloc[j-2] == 4)):
      C5_dihedrals.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-3]
      C5_dihedrals.loc[i, 'atom_id2'] = C5.atom_id.iloc[j-2]
      C5_dihedrals.loc[i, 'atom_id3'] = C5.atom_id.iloc[j-1]
      C5_dihedrals.loc[i, 'atom_id4'] = C5.atom_id.iloc[j]
      i = i+1
    elif (C5.atom_type.iloc[j] == C5.atom_type.iloc[j-1] == C5.atom_type.iloc[j-2] == C5.atom_type.iloc[j-3] == 4):
      C5_dihedrals.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-3]
      C5_dihedrals.loc[i, 'atom_id2'] = C5.atom_id.iloc[j-2]
      C5_dihedrals.loc[i, 'atom_id3'] = C5.atom_id.iloc[j-1]
      C5_dihedrals.loc[i, 'atom_id4'] = C5.atom_id.iloc[j]
      i = i+1
    elif ((C5.atom_type.iloc[j-3] == 3) and (C5.atom_type.iloc[j] == C5.atom_type.iloc[j-1] == C5.atom_type.iloc[j-2] == 4)):
      C5_dihedrals.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-3]
      C5_dihedrals.loc[i, 'atom_id2'] = C5.atom_id.iloc[j-2]
      C5_dihedrals.loc[i, 'atom_id3'] = C5.atom_id.iloc[j-1]
      C5_dihedrals.loc[i, 'atom_id4'] = C5.atom_id.iloc[j]
      i = i+1
    elif ((C5.atom_type.iloc[j] == 3) and (C5.atom_type.iloc[j-1] == C5.atom_type.iloc[j-2] == C5.atom_type.iloc[j-3] == 4)):
      C5_dihedrals.loc[i, 'atom_id1'] = C5.atom_id.iloc[j-3]
      C5_dihedrals.loc[i, 'atom_id2'] = C5.atom_id.iloc[j-2]
      C5_dihedrals.loc[i, 'atom_id3'] = C5.atom_id.iloc[j-1]
      C5_dihedrals.loc[i, 'atom_id4'] = C5.atom_id.iloc[j]
      i = i+1

dihed_id = list(range(1, C5_dihedrals.shape[0]+1))
C5_dihedrals.dihedral_id = dihed_id

C5_dihedrals = C5_dihedrals.assign(dihedral_type = 1)

In [ ]:
C5_dihedrals_copy = C5_dihedrals.copy()

C5_dihedrals_copy.dihedral_id = C5_dihedrals_copy.dihedral_id+C5_dihedrals.shape[0]

C5_dihedrals_copy.atom_id1 = C5_dihedrals_copy.atom_id1+C5_shape
C5_dihedrals_copy.atom_id2 = C5_dihedrals_copy.atom_id2+C5_shape 
C5_dihedrals_copy.atom_id3 = C5_dihedrals_copy.atom_id3+C5_shape
C5_dihedrals_copy.atom_id4 = C5_dihedrals_copy.atom_id4+C5_shape

In [ ]:
C5_dihedrals = pd.concat([C5_dihedrals, C5_dihedrals_copy]).reset_index(drop = True)

del C5_dihedrals_copy

In [ ]:
# dihedral type 1 - CH3-CH2-CH2-CH3  CH2-CH2-CH2-CH2  CH3-CH2-CH2-CH2

C7_dihedrals = pd.DataFrame(columns = ['dihedral_id', 'dihedral_type', 'atom_id1', 'atom_id2', 'atom_id3', 'atom_id4'])

i = 0

for j in range(0, C7_shape):
  if (C7.molecule_id.iloc[j] == C7.molecule_id.iloc[j-1] == C7.molecule_id.iloc[j-2] == C7.molecule_id.iloc[j-3]):
    if ((C7.atom_type.iloc[j] == C7.atom_type.iloc[j-3] == 3) and (C7.atom_type.iloc[j-1] == C7.atom_type.iloc[j-2] == 4)):
      C7_dihedrals.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-3]
      C7_dihedrals.loc[i, 'atom_id2'] = C7.atom_id.iloc[j-2]
      C7_dihedrals.loc[i, 'atom_id3'] = C7.atom_id.iloc[j-1]
      C7_dihedrals.loc[i, 'atom_id4'] = C7.atom_id.iloc[j]
      i = i+1
    elif (C7.atom_type.iloc[j] == C7.atom_type.iloc[j-1] == C7.atom_type.iloc[j-2] == C7.atom_type.iloc[j-3] == 4):
      C7_dihedrals.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-3]
      C7_dihedrals.loc[i, 'atom_id2'] = C7.atom_id.iloc[j-2]
      C7_dihedrals.loc[i, 'atom_id3'] = C7.atom_id.iloc[j-1]
      C7_dihedrals.loc[i, 'atom_id4'] = C7.atom_id.iloc[j]
      i = i+1
    elif ((C7.atom_type.iloc[j-3] == 3) and (C7.atom_type.iloc[j] == C7.atom_type.iloc[j-1] == C7.atom_type.iloc[j-2] == 4)):
      C7_dihedrals.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-3]
      C7_dihedrals.loc[i, 'atom_id2'] = C7.atom_id.iloc[j-2]
      C7_dihedrals.loc[i, 'atom_id3'] = C7.atom_id.iloc[j-1]
      C7_dihedrals.loc[i, 'atom_id4'] = C7.atom_id.iloc[j]
      i = i+1
    elif ((C7.atom_type.iloc[j] == 3) and (C7.atom_type.iloc[j-1] == C7.atom_type.iloc[j-2] == C7.atom_type.iloc[j-3] == 4)):
      C7_dihedrals.loc[i, 'atom_id1'] = C7.atom_id.iloc[j-3]
      C7_dihedrals.loc[i, 'atom_id2'] = C7.atom_id.iloc[j-2]
      C7_dihedrals.loc[i, 'atom_id3'] = C7.atom_id.iloc[j-1]
      C7_dihedrals.loc[i, 'atom_id4'] = C7.atom_id.iloc[j]
      i = i+1

dihed_id = list(range(1, C7_dihedrals.shape[0]+1))
C7_dihedrals.dihedral_id = dihed_id
C7_dihedrals.dihedral_id = C7_dihedrals.dihedral_id + C5_dihedrals.dihedral_id.max()

C7_dihedrals = C7_dihedrals.assign(dihedral_type = 1)

In [ ]:
C7_dihedrals_copy = C7_dihedrals.copy()

C7_dihedrals_copy.dihedral_id = C7_dihedrals_copy.dihedral_id+C7_dihedrals.shape[0]

C7_dihedrals_copy.atom_id1 = C7_dihedrals_copy.atom_id1+C7_shape
C7_dihedrals_copy.atom_id2 = C7_dihedrals_copy.atom_id2+C7_shape 
C7_dihedrals_copy.atom_id3 = C7_dihedrals_copy.atom_id3+C7_shape
C7_dihedrals_copy.atom_id4 = C7_dihedrals_copy.atom_id4+C7_shape

In [ ]:
C7_dihedrals = pd.concat([C7_dihedrals, C7_dihedrals_copy]).reset_index(drop = True)

del C7_dihedrals_copy

In [ ]:
# dihedral type 1 - CH3-CH2-CH2-CH3  CH2-CH2-CH2-CH2  CH3-CH2-CH2-CH2

C13_dihedrals = pd.DataFrame(columns = ['dihedral_id', 'dihedral_type', 'atom_id1', 'atom_id2', 'atom_id3', 'atom_id4'])

i = 0

for j in range(0, C13_shape):
  if (C13.molecule_id.iloc[j] == C13.molecule_id.iloc[j-1] == C13.molecule_id.iloc[j-2] == C13.molecule_id.iloc[j-3]):
    if ((C13.atom_type.iloc[j] == C13.atom_type.iloc[j-3] == 3) and (C13.atom_type.iloc[j-1] == C13.atom_type.iloc[j-2] == 4)):
      C13_dihedrals.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-3]
      C13_dihedrals.loc[i, 'atom_id2'] = C13.atom_id.iloc[j-2]
      C13_dihedrals.loc[i, 'atom_id3'] = C13.atom_id.iloc[j-1]
      C13_dihedrals.loc[i, 'atom_id4'] = C13.atom_id.iloc[j]
      i = i+1
    elif (C13.atom_type.iloc[j] == C13.atom_type.iloc[j-1] == C13.atom_type.iloc[j-2] == C13.atom_type.iloc[j-3] == 4):
      C13_dihedrals.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-3]
      C13_dihedrals.loc[i, 'atom_id2'] = C13.atom_id.iloc[j-2]
      C13_dihedrals.loc[i, 'atom_id3'] = C13.atom_id.iloc[j-1]
      C13_dihedrals.loc[i, 'atom_id4'] = C13.atom_id.iloc[j]
      i = i+1
    elif ((C13.atom_type.iloc[j-3] == 3) and (C13.atom_type.iloc[j] == C13.atom_type.iloc[j-1] == C13.atom_type.iloc[j-2] == 4)):
      C13_dihedrals.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-3]
      C13_dihedrals.loc[i, 'atom_id2'] = C13.atom_id.iloc[j-2]
      C13_dihedrals.loc[i, 'atom_id3'] = C13.atom_id.iloc[j-1]
      C13_dihedrals.loc[i, 'atom_id4'] = C13.atom_id.iloc[j]
      i = i+1
    elif ((C13.atom_type.iloc[j] == 3) and (C13.atom_type.iloc[j-1] == C13.atom_type.iloc[j-2] == C13.atom_type.iloc[j-3] == 4)):
      C13_dihedrals.loc[i, 'atom_id1'] = C13.atom_id.iloc[j-3]
      C13_dihedrals.loc[i, 'atom_id2'] = C13.atom_id.iloc[j-2]
      C13_dihedrals.loc[i, 'atom_id3'] = C13.atom_id.iloc[j-1]
      C13_dihedrals.loc[i, 'atom_id4'] = C13.atom_id.iloc[j]
      i = i+1

dihed_id = list(range(1, C13_dihedrals.shape[0]+1))
C13_dihedrals.dihedral_id = dihed_id
C13_dihedrals.dihedral_id = C13_dihedrals.dihedral_id + C7_dihedrals.dihedral_id.max()

C13_dihedrals = C13_dihedrals.assign(dihedral_type = 1)

In [ ]:
C13_dihedrals_copy = C13_dihedrals.copy()

C13_dihedrals_copy.dihedral_id = C13_dihedrals_copy.dihedral_id+C13_dihedrals.shape[0]

C13_dihedrals_copy.atom_id1 = C13_dihedrals_copy.atom_id1+C13_shape
C13_dihedrals_copy.atom_id2 = C13_dihedrals_copy.atom_id2+C13_shape 
C13_dihedrals_copy.atom_id3 = C13_dihedrals_copy.atom_id3+C13_shape
C13_dihedrals_copy.atom_id4 = C13_dihedrals_copy.atom_id4+C13_shape

In [ ]:
C13_dihedrals = pd.concat([C13_dihedrals, C13_dihedrals_copy]).reset_index(drop = True)

del C13_dihedrals_copy

In [ ]:
C_dihedrals = pd.concat([C5_dihedrals, C7_dihedrals]).reset_index(drop = True)
C_dihedrals = pd.concat([C_dihedrals, C13_dihedrals]).reset_index(drop = True)

del C5_dihedrals, C7_dihedrals, C13_dihedrals

In [ ]:
C = pd.concat([C1, C1_copy]).reset_index(drop = True)
C = pd.concat([C, C2]).reset_index(drop = True)
C = pd.concat([C, C2_copy]).reset_index(drop = True)
C = pd.concat([C, C5]).reset_index(drop = True)
C = pd.concat([C, C5_copy]).reset_index(drop = True)
C = pd.concat([C, C7]).reset_index(drop = True)
C = pd.concat([C, C7_copy]).reset_index(drop = True)
C = pd.concat([C, C13]).reset_index(drop = True)
C = pd.concat([C, C13_copy]).reset_index(drop = True)

del C1, C1_copy, C2, C2_copy, C5, C5_copy, C7, C7_copy, C13, C13_copy

C

In [ ]:
C.to_csv(C_out_filename, index = False)
C_bonds.to_csv(bonds_out_filename, index = False)
C_angles.to_csv(angles_out_filename, index = False)
C_dihedrals.to_csv(dihedrals_out_filename, index = False)

In [ ]:
z = C.z_pos
x = C.x_pos
y = C.y_pos

ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(x, y, z, color = "green")
plt.title("simple 3D scatter plot")
 
# show plot
plt.show()